In [6]:
from IPython.core.display import HTML
display(HTML("<style>.container { width:95% !important; }</style>"))   

Author: Andrew Tarzia

Date Created: 12 Feb 2018

License:

# Notebook for screening BRENDA database post pI screening

TODO:
- take BRENDA files and split into property specific files using code in python script
- filter out proteins based on BRENDA information
- collect product and substrate names
- output
- turn notebook into presentation of process for an example.
- move to scripts
    - generalise to running in current working dir
    - add license

In [7]:
import os
import pandas as pd
import glob
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import time
import numpy as np
from shutil import copyfile

In [8]:
# my imports
source_dir = '/home/atarzia/psp/psp_source/'
os.chdir(source_dir)
import brenda_IO

# Notes:


In [9]:
# get parameters
source_dir = '/home/atarzia/psp/psp_source/'
os.chdir(source_dir)
%run ./parameter_file.py
param_dict = get_parameters()

# input parameters
database_directory = '/home/atarzia/psp/sequence_db/test_dataset/'
database_directory = '/home/atarzia/psp/sequence_db/linb_dataset/'

os.chdir(database_directory)
# get input FASTA file names
database_names = []
for i in glob.glob("*fasta"):
    if "_mod" not in i:
        database_names.append(i)

# output CSV file
redo_pi = True
if redo_pi is True:
    string = ''
    for i in param_dict['out_columns_br']:
        if i == param_dict['out_columns_br'][-1]:
            string += i
        else:
            string += i+','
    string += '\n'
    with open(param_dict['out_CSV_br'], 'w') as f:
        f.write(string)

In [10]:
print('=====================================================================')
print('                     Screening enzymes with Brenda!                  ')
print('=====================================================================')
# read in pI output file
pi_data = pd.read_csv(param_dict['out_CSV_pi'], index_col=False)
# only keep those that pass the pI test
pi_passed_data = pi_data[pi_data['pi'] < param_dict['cutoff_pi']]
print('--- ', 
      len(pi_passed_data), 'proteins out of', len(pi_data), 
      'remain, OR', int(len(pi_passed_data)/len(pi_data)*100),
      "% ---")
# for each FASTA file
count_done = 0
total_start_time = time.time()
for i, row in pi_passed_data.iterrows():
    EC = row['EC.code'].replace(" ", "")
    EC_ = EC.replace(".", "_")
    # get brenda data file
    br_datafile = 'brenda_download_'+EC_+'.txt'
    if os.path.isfile(br_datafile) is False:
        src = param_dict['BRENDA_DB_loc']+br_datafile
        des = database_directory+br_datafile
        copyfile(src, des)
    print(EC)
    print(br_datafile)
    print(row)
    # convert br_datafile into dictionary
    br_symbols, br_data = brenda_IO.get_brenda_dict(br_datafile)
    
    
    
    break
    
print('--- finished %s proteins in %s seconds ---' % (count_done, '{0:.2f}'.format(time.time() - total_start_time)))
print('=====================================================================')
print('                              Complete!                              ')
print('=====================================================================')

                     Screening enzymes with Brenda!                  
---  2858 proteins out of 4524 remain, OR 63 % ---
4.5.1.1
brenda_download_4_5_1_1.txt
fasta_file      4_5_1_1__BRENDA_sequences.fasta
acc.code                                Q93112 
organism                 DDT-dehydrochlorinase 
EC.code                                4.5.1.1 
species                      Anopheles gambiae 
note                                 Swiss-Prot
pi                                         5.84
modification                                  0
category                                      0
Name: 0, dtype: object
--- finished 0 proteins in 0.00 seconds ---
                              Complete!                              


In [12]:
br_data

{'AC': ['#4# cysteinylglycine (#4# activates <2>) <2>',
  '#4# GSH (#4# required <1,3,4>; #4# Km: 0.25 mM <1>; #4# stabilizestetrameric enzyme form <4>) <1,3,4>',
  '#4# cysteinylglycine (#4# activates <2>) <2>',
  '#4# GSH (#4# required <1,3,4>; #4# Km: 0.25 mM <1>; #4# stabilizestetrameric enzyme form <4>) <1,3,4>'],
 'AP': ['#5,7,8# biotechnology (#7# positive correlation of enzyme activity andresistance against DDT <9>; #5# positive correlation of enzyme activity'],
 'CF': ['#3# glutathione <11>'],
 'CL': ['#6# <7>'],
 'ID': ['4.5.1.1'],
 'IN': ['#2# guanidine hydrochloride (#2# complete inhibition at 1 M <10>) <10>',
  '#2# SDS (#2# 78% loss of activity at 5% (w/v) <10>) <10>',
  '#2# Urea (#2# 98% loss of activity at 10 mM <10>) <10>',
  '#3# permethrin (#3# uncompetitive inhibitor of isoform GSTE2-2 <11>)<11>',
  '#4# iodoacetate (#4# inhibition to a minor degree, completelyreversible <2>) <2>',
  '#4# p-chloromercuribenzoate (#4# i.e. PCMB.1,1,1-trichloro-2,2-bis(4-chlorophenyl